In [ ]:
!pip install tqdm==4.36.1

     |████████████████████████████████| 61kB 2.3MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [ ]:
!pip install testfixtures

     |████████████████████████████████| 92kB 3.1MB/s 


In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 4.9MB/s 
     |████████████████████████████████| 245kB 51.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81705 sha256=d2bea4494607f458d65ede82366c07736f28a0b7394b53fd28f05063c2f4f2fe
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
!pip install textsearch

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving essays.csv to essays.csv


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier, LogisticRegressionCV
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from gensim.models.doc2vec import TaggedDocument
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
import xgboost as xgb
import re
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import contractions
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import multiprocessing

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
cores = multiprocessing.cpu_count()

In [ ]:
#Initializing Classifiers

xgbc = xgb.XGBClassifier(random_state=1,learning_rate=0.01)
logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=750, solver='lbfgs')
lgcv = LogisticRegressionCV(cv=5, random_state=0, multi_class='ovr')
sgd = SGDClassifier(max_iter=1000, tol=1e-3, loss='log')
pac = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3)
rc = RidgeClassifier()
svc = SVC(gamma='auto', probability=True)

In [ ]:
#DATASET LINK --> https://github.com/SenticNet/personality-detection/blob/master/essays.csv
df = pd.read_csv('essays.csv')

**Basic Exploratory Data Analysis**

In [ ]:
df.head(10)

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997504851,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
1,1997605191,"Well, here we go with the stream of consciousn...",n,n,y,n,n
2,1997687252,An open keyboard and buttons to push. The thin...,n,y,n,y,y
3,1997568848,I can't believe it! It's really happening! M...,y,n,y,y,n
4,1997688160,"Well, here I go with the good old stream of co...",y,n,y,n,y
5,1997722902,Today. Had to turn the music down. Today I wen...,y,n,y,n,y
6,1997724708,Stream of consciousness. What should I write a...,n,n,y,n,n
7,1997724794,The RTF305 Usenet site is a piece of garbage! ...,n,n,n,y,y
8,1997628043,I'm really unsure about this assignment becaus...,y,y,n,y,y
9,1997708036,Today was a tough day for me. I can't believed...,y,y,y,y,n


In [ ]:
df.shape

(2467, 7)

In [ ]:
df.index = range(2467)
df['TEXT'].apply(lambda x: len(x.split(' '))).sum()

1634170

Converting 'n'->0 and 'y'->1





In [ ]:
df['cEXT'] = df['cEXT'].apply(lambda x: 0 if x=='n' else 1)
df['cNEU'] = df['cNEU'].apply(lambda x: 0 if x=='n' else 1)
df['cAGR'] = df['cAGR'].apply(lambda x: 0 if x=='n' else 1)
df['cCON'] = df['cCON'].apply(lambda x: 0 if x=='n' else 1)
df['cOPN'] = df['cOPN'].apply(lambda x: 0 if x=='n' else 1)

In [ ]:
df.head(10)

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997504851,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,1997605191,"Well, here we go with the stream of consciousn...",0,0,1,0,0
2,1997687252,An open keyboard and buttons to push. The thin...,0,1,0,1,1
3,1997568848,I can't believe it! It's really happening! M...,1,0,1,1,0
4,1997688160,"Well, here I go with the good old stream of co...",1,0,1,0,1
5,1997722902,Today. Had to turn the music down. Today I wen...,1,0,1,0,1
6,1997724708,Stream of consciousness. What should I write a...,0,0,1,0,0
7,1997724794,The RTF305 Usenet site is a piece of garbage! ...,0,0,0,1,1
8,1997628043,I'm really unsure about this assignment becaus...,1,1,0,1,1
9,1997708036,Today was a tough day for me. I can't believed...,1,1,1,1,0


In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

In [ ]:
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = replace_contractions(text)
    return text
df['TEXT'] = df['TEXT'].apply(cleanText)

In [ ]:
def normalize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 3 or word.isdigit() or word in stopwords.words('english'):
                continue
            else:
                lemmatizer = WordNetLemmatizer()
                new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
                lemma = lemmatizer.lemmatize(new_word, pos='v')
                tokens.append(lemma.lower())
    return tokens

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

**We will try and predict the personality trait of Extroversion(cEXT) for the given dataset**

In [ ]:
# Training and Test Documents for Extroversion
train_tagged_ext = train.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cEXT]), axis=1)
test_tagged_ext = test.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cEXT]), axis=1)

In [ ]:
train_tagged_ext.values[30]

TaggedDocument(words=['feel', 'really', 'crappy', 'today', 'get', 'enough', 'sleep', 'last', 'night', 'think', 'part', 'texas', 'loss', 'saturday', 'especially', 'way', 'loss', 'least', 'cowboys', 'last', 'night', 'far', 'home', 'sick', 'might', 'roommate', 'go', 'high', 'school', 'also', 'able', 'talk', 'parent', 'lot', 'instant', 'chat', 'mom', 'wnts', 'get', 'web', 'cam', 'actually', 'see', 'think', 'hard', 'time', 'leave', 'first', 'child', 'leave', 'home', 'network', 'really', 'cool', 'connect', 'rat', 'really', 'fast', 'crummy', 'modem', 'home', 'dad', 'go', 'get', 'dsl', 'live', 'far', 'central', 'office', 'suck', 'really', 'hungry', 'hungry', 'morning', 'halfway', 'class', 'stomach', 'start', 'hurt', 'probably', 'use', 'sleep', 'late', 'eat', 'breakfast', 'min', 'take', 'english', 'year', 'good', 'like', 'write', 'paper', 'much', 'might', 'write', 'paper', 'feel', 'perfectly', 'valid', 'turn', 'wrong', 'yet', 'put', 'much', 'work', 'min', 'milk', 'taste', 'pretty', 'good', 'use

**Distributed Bag of Words Model(PV-DBOW) for Doc2Vec**

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=230, negative=5, hs=1, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged_ext.values)])

100%|██████████| 1973/1973 [00:00<00:00, 349112.46it/s]


In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged_ext.values)]), total_examples=len(train_tagged_ext.values), epochs=1)
    model_dbow.alpha -= 0.02
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1973/1973 [00:00<00:00, 1424575.97it/s]


CPU times: user 1min 13s, sys: 641 ms, total: 1min 14s
Wall time: 39.5 s


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
#Creating Train and Test Paragraph Vectors for Distributed Bag of Words Model
Y_train_ext, X_train_ext = vec_for_learning(model_dbow, train_tagged_ext)
Y_test_ext, X_test_ext = vec_for_learning(model_dbow, test_tagged_ext)

In [ ]:
# Results for Logistic Regression 

logreg.fit(X_train_ext, Y_train_ext)
Y_lg_pred_ext = logreg.predict(X_test_ext)
print('Testing accuracy while using Logistic Regression %s' % accuracy_score(Y_test_ext, Y_lg_pred_ext))
print('Testing F1 score while using Logistic Regression: {}'.format(f1_score(Y_test_ext, Y_lg_pred_ext, average='weighted')))

Testing accuracy while using Logistic Regression 0.5566801619433198
Testing F1 score while using Logistic Regression: 0.5571871662466455


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for Logistic Regression with CV

lgcv.fit(X_train_ext, Y_train_ext)
Y_lgcv_pred_ext = lgcv.predict(X_test_ext)
print('Testing accuracy while using Logistic Regression with CV %s' % accuracy_score(Y_test_ext, Y_lgcv_pred_ext))
print('Testing F1 score while using Logistic Regression with CV: {}'.format(f1_score(Y_test_ext, Y_lgcv_pred_ext, average='weighted')))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

Testing accuracy while using Logistic Regression with CV 0.548582995951417
Testing F1 score while using Logistic Regression with CV: 0.5492327245675112


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for SGD Classifier

sgd.fit(X_train_ext, Y_train_ext)
Y_sgd_pred_ext = sgd.predict(X_test_ext)
print('Testing accuracy while using SGD Classifier %s' % accuracy_score(Y_test_ext, Y_sgd_pred_ext))
print('Testing F1 score while using SGD Classifier: {}'.format(f1_score(Y_test_ext, Y_sgd_pred_ext, average='weighted')))

Testing accuracy while using SGD Classifier 0.48785425101214575
Testing F1 score while using SGD Classifier: 0.4588469123073096


In [ ]:
# Results for Passive Aggressive Classifier

pac.fit(X_train_ext, Y_train_ext)
Y_pac_pred_ext = pac.predict(X_test_ext)
print('Testing accuracy while using Passive Aggressive Classifier %s' % accuracy_score(Y_test_ext, Y_pac_pred_ext))
print('Testing F1 score while using Passive Aggressive Classifier: {}'.format(f1_score(Y_test_ext, Y_pac_pred_ext, average='weighted')))

Testing accuracy while using Passive Aggressive Classifier 0.48582995951417
Testing F1 score while using Passive Aggressive Classifier: 0.4572420877460796


In [ ]:
# Results for Ridge Classifier

rc.fit(X_train_ext, Y_train_ext)
Y_rc_pred_ext = rc.predict(X_test_ext)
print('Testing accuracy while using Ridge Classifier %s' % accuracy_score(Y_test_ext, Y_rc_pred_ext))
print('Testing F1 score while using Ridge Classifier: {}'.format(f1_score(Y_test_ext, Y_rc_pred_ext, average='weighted')))

Testing accuracy while using Ridge Classifier 0.5344129554655871
Testing F1 score while using Ridge Classifier: 0.5348022167651159


In [ ]:
# Results for Support Vector Classifier

svc.fit(X_train_ext, Y_train_ext)
Y_svc_pred_ext = svc.predict(X_test_ext)
print('Testing accuracy while using Support Vector Classifier %s' % accuracy_score(Y_test_ext, Y_svc_pred_ext))
print('Testing F1 score while using Support Vector Classifier: {}'.format(f1_score(Y_test_ext, Y_svc_pred_ext, average='weighted')))

Testing accuracy while using Support Vector Classifier 0.562753036437247
Testing F1 score while using Support Vector Classifier: 0.5364706444623204


In [ ]:
# Results for XGBoost Algorithm
xgbc.fit(np.array(X_train_ext), Y_train_ext)
xgb_predict_ext = xgbc.predict(np.array(X_test_ext))

print('Testing accuracy while using XGBoost Algorithm %s' % accuracy_score(Y_test_ext,xgb_predict_ext))
print('Testing F1 score while using XGBoost Algorithm: {}'.format(f1_score(Y_test_ext, xgb_predict_ext, average='weighted')))

Testing accuracy while using XGBoost Algorithm 0.5506072874493927
Testing F1 score while using XGBoost Algorithm: 0.5445557772398204


In [ ]:
hard_voting = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='hard')
soft_voting = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='soft')

In [ ]:
hard_voting.fit(np.array(X_train_ext), Y_train_ext)
soft_voting.fit(np.array(X_train_ext), Y_train_ext)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=100000.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=750,
                                                 multi_class='warn', n_jobs=1,
                                                 penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('SGD',
                              SGDClassifier(alpha=0.0001, average=False,
                                            class_weight=None,
                                            early_stop...
                                  random_state=None, shri

In [ ]:
Y_hard_voting = hard_voting.predict(np.array(X_test_ext))
Y_soft_voting = soft_voting.predict(np.array(X_test_ext))

In [ ]:
print('Testing accuracy while using Ensemble Model with Hard Voting%s' % accuracy_score(Y_test_ext,Y_hard_voting))
print('Testing F1 score while using Ensemble Model with Hard Voting: {}'.format(f1_score(Y_test_ext, Y_hard_voting, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting0.5566801619433198
Testing F1 score while using Ensemble Model with Hard Voting: 0.5571890017451003


In [ ]:
print('Testing accuracy while using Ensemble Model with Hard Voting%s' % accuracy_score(Y_test_ext,Y_soft_voting))
print('Testing F1 score while using Ensemble Model with Hard Voting: {}'.format(f1_score(Y_test_ext, Y_soft_voting, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting0.548582995951417
Testing F1 score while using Ensemble Model with Hard Voting: 0.5452266874180666


**Out of all the models shown above, Support Vector Classifier has the best Accuracy and F1-Score, 56.27% and 53.64% respectively. However, for cEXT, the State-of-the-Art model has an accuracy of 58.09%. Thus, our model fails to match the SOTA Accuracy, by 1.82%.**

**We will try and predict the personality trait of Neuroticism(cNEU) for the given dataset**

In [ ]:
# Training and Test Documents for Neuroticism
train_tagged_neu = train.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cNEU]), axis=1)
test_tagged_neu = test.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cNEU]), axis=1)

In [ ]:
train_tagged_neu.values[30]

TaggedDocument(words=['feel', 'really', 'crappy', 'today', 'get', 'enough', 'sleep', 'last', 'night', 'think', 'part', 'texas', 'loss', 'saturday', 'especially', 'way', 'loss', 'least', 'cowboys', 'last', 'night', 'far', 'home', 'sick', 'might', 'roommate', 'go', 'high', 'school', 'also', 'able', 'talk', 'parent', 'lot', 'instant', 'chat', 'mom', 'wnts', 'get', 'web', 'cam', 'actually', 'see', 'think', 'hard', 'time', 'leave', 'first', 'child', 'leave', 'home', 'network', 'really', 'cool', 'connect', 'rat', 'really', 'fast', 'crummy', 'modem', 'home', 'dad', 'go', 'get', 'dsl', 'live', 'far', 'central', 'office', 'suck', 'really', 'hungry', 'hungry', 'morning', 'halfway', 'class', 'stomach', 'start', 'hurt', 'probably', 'use', 'sleep', 'late', 'eat', 'breakfast', 'min', 'take', 'english', 'year', 'good', 'like', 'write', 'paper', 'much', 'might', 'write', 'paper', 'feel', 'perfectly', 'valid', 'turn', 'wrong', 'yet', 'put', 'much', 'work', 'min', 'milk', 'taste', 'pretty', 'good', 'use

In [ ]:
neu_model_dbow = Doc2Vec(dm=0, vector_size=230, negative=5, hs=1, min_count=2, sample = 0, workers=cores)
neu_model_dbow.build_vocab([x for x in tqdm(train_tagged_neu.values)])

100%|██████████| 1973/1973 [00:00<00:00, 1435946.87it/s]


In [ ]:
%%time
for epoch in range(30):
    neu_model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged_neu.values)]), total_examples=len(train_tagged_neu.values), epochs=1)
    neu_model_dbow.alpha -= 0.02
    neu_model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1973/1973 [00:00<00:00, 1425066.61it/s]


CPU times: user 1min 13s, sys: 745 ms, total: 1min 14s
Wall time: 39.5 s


In [ ]:
#Creating Train and Test Paragraph Vectors for Distributed Bag of Words Model
Y_train_neu, X_train_neu = vec_for_learning(neu_model_dbow, train_tagged_neu)
Y_test_neu, X_test_neu = vec_for_learning(neu_model_dbow, test_tagged_neu)

In [ ]:
# Results for Logistic Regression 

logreg.fit(X_train_neu, Y_train_neu)
Y_lg_pred_neu = logreg.predict(X_test_neu)
print('Testing accuracy while using Logistic Regression %s' % accuracy_score(Y_test_neu, Y_lg_pred_neu))
print('Testing F1 score while using Logistic Regression: {}'.format(f1_score(Y_test_neu, Y_lg_pred_neu, average='weighted')))

Testing accuracy while using Logistic Regression 0.4676113360323887
Testing F1 score while using Logistic Regression: 0.4676658762278491


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for Logistic Regression with CV

lgcv.fit(X_train_neu, Y_train_neu)
Y_lgcv_pred_neu = lgcv.predict(X_test_neu)
print('Testing accuracy while using Logistic Regression with CV %s' % accuracy_score(Y_test_neu, Y_lgcv_pred_neu))
print('Testing F1 score while using Logistic Regression with CV: {}'.format(f1_score(Y_test_neu, Y_lgcv_pred_neu, average='weighted')))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

Testing accuracy while using Logistic Regression with CV 0.5121457489878543
Testing F1 score while using Logistic Regression with CV: 0.5106826685910089


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for SGD Classifier

sgd.fit(X_train_neu, Y_train_neu)
Y_sgd_pred_neu = sgd.predict(X_test_neu)
print('Testing accuracy while using SGD Classifier %s' % accuracy_score(Y_test_neu, Y_sgd_pred_neu))
print('Testing F1 score while using SGD Classifier: {}'.format(f1_score(Y_test_neu, Y_sgd_pred_neu, average='weighted')))

Testing accuracy while using SGD Classifier 0.4898785425101215
Testing F1 score while using SGD Classifier: 0.48765009009542537


In [ ]:
# Results for Passive Aggressive Classifier

pac.fit(X_train_neu, Y_train_neu)
Y_pac_pred_neu = pac.predict(X_test_neu)
print('Testing accuracy while using Passive Aggressive Classifier %s' % accuracy_score(Y_test_neu, Y_pac_pred_neu))
print('Testing F1 score while using Passive Aggressive Classifier: {}'.format(f1_score(Y_test_neu, Y_pac_pred_neu, average='weighted')))

Testing accuracy while using Passive Aggressive Classifier 0.4898785425101215
Testing F1 score while using Passive Aggressive Classifier: 0.48765009009542537


In [ ]:
# Results for Ridge Classifier

rc.fit(X_train_neu, Y_train_neu)
Y_rc_pred_neu = rc.predict(X_test_neu)
print('Testing accuracy while using Ridge Classifier %s' % accuracy_score(Y_test_neu, Y_rc_pred_neu))
print('Testing F1 score while using Ridge Classifier: {}'.format(f1_score(Y_test_neu, Y_rc_pred_neu, average='weighted')))

Testing accuracy while using Ridge Classifier 0.5222672064777328
Testing F1 score while using Ridge Classifier: 0.5213739990176843


In [ ]:
# Results for Support Vector Classifier

svc.fit(X_train_neu, Y_train_neu)
Y_svc_pred_neu = svc.predict(X_test_neu)
print('Testing accuracy while using Support Vector Classifier %s' % accuracy_score(Y_test_neu, Y_svc_pred_neu))
print('Testing F1 score while using Support Vector Classifier: {}'.format(f1_score(Y_test_neu, Y_svc_pred_neu, average='weighted')))

Testing accuracy while using Support Vector Classifier 0.4898785425101215
Testing F1 score while using Support Vector Classifier: 0.40475774059197556


In [ ]:
# Results for XGBoost Algorithm

xgbc.fit(np.array(X_train_neu), Y_train_neu)
xgb_predict_neu = xgbc.predict(np.array(X_test_neu))
print('Testing accuracy while using XGBoost Algorithm %s' % accuracy_score(Y_test_neu,xgb_predict_neu))
print('Testing F1 score while using XGBoost Algorithm: {}'.format(f1_score(Y_test_neu, xgb_predict_neu, average='weighted')))

Testing accuracy while using XGBoost Algorithm 0.5161943319838057
Testing F1 score while using XGBoost Algorithm: 0.511940302713243


In [ ]:
hard_voting_neu = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='hard')
soft_voting_neu = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='soft')

In [ ]:
hard_voting_neu.fit(np.array(X_train_neu), Y_train_neu)
soft_voting_neu.fit(np.array(X_train_neu), Y_train_neu)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=100000.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=750,
                                                 multi_class='warn', n_jobs=1,
                                                 penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('SGD',
                              SGDClassifier(alpha=0.0001, average=False,
                                            class_weight=None,
                                            early_stop...
                                  random_state=None, shri

In [ ]:
Y_hard_voting_neu = hard_voting_neu.predict(np.array(X_test_neu))
Y_soft_voting_neu = soft_voting_neu.predict(np.array(X_test_neu))

In [ ]:
print('Testing accuracy while using Ensemble Model with Hard Voting %s' % accuracy_score(Y_test_neu,Y_hard_voting_neu))
print('Testing F1 score while using Ensemble Model with Hard Voting: {}'.format(f1_score(Y_test_neu, Y_hard_voting_neu, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting0.5060728744939271
Testing F1 score while using Ensemble Model with Hard Voting: 0.4900025182573659


In [ ]:
print('Testing accuracy while using Ensemble Model with Soft Voting %s' % accuracy_score(Y_test_neu,Y_soft_voting_neu))
print('Testing F1 score while using Ensemble Model with S Voting: {}'.format(f1_score(Y_test_neu, Y_soft_voting_neu, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting0.4959514170040486
Testing F1 score while using Ensemble Model with Hard Voting: 0.4932895745068139


**Out of all the models shown above, Ridge Classifier has the best Accuracy and F1-Score, 52.22% and 52.12% respectively. However, for cNEU, the State-of-the-Art model has an accuracy of 59.38%. Thus, our model fails to match the SOTA Accuracy, by 7.16%.**

**We will try and predict the personality trait of Aggression(cAGR) for the given dataset**

In [ ]:
# Training and Test Documents for Agression
train_tagged_agr = train.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cAGR]), axis=1)
test_tagged_agr = test.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cAGR]), axis=1)

In [ ]:
train_tagged_agr.values[30]

TaggedDocument(words=['feel', 'really', 'crappy', 'today', 'get', 'enough', 'sleep', 'last', 'night', 'think', 'part', 'texas', 'loss', 'saturday', 'especially', 'way', 'loss', 'least', 'cowboys', 'last', 'night', 'far', 'home', 'sick', 'might', 'roommate', 'go', 'high', 'school', 'also', 'able', 'talk', 'parent', 'lot', 'instant', 'chat', 'mom', 'wnts', 'get', 'web', 'cam', 'actually', 'see', 'think', 'hard', 'time', 'leave', 'first', 'child', 'leave', 'home', 'the', 'network', 'really', 'cool', 'the', 'connect', 'rat', 'really', 'fast', 'crummy', 'modem', 'home', 'dad', 'go', 'get', 'dsl', 'live', 'far', 'central', 'office', 'suck', 'really', 'hungry', 'hungry', 'morning', 'halfway', 'class', 'stomach', 'start', 'hurt', 'probably', 'use', 'sleep', 'late', 'eat', 'breakfast', 'min', 'take', 'english', 'year', 'good', 'like', 'write', 'paper', 'much', 'you', 'might', 'write', 'paper', 'feel', 'perfectly', 'valid', 'turn', 'wrong', 'yet', 'put', 'much', 'work', 'min', 'milk', 'taste', '

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=230, negative=5, hs=1, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged_agr.values)])

100%|██████████| 1973/1973 [00:00<00:00, 323067.02it/s]


In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged_agr.values)]), total_examples=len(train_tagged_agr.values), epochs=1)
    model_dbow.alpha -= 0.02
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1973/1973 [00:00<00:00, 948356.84it/s]


CPU times: user 1min 6s, sys: 479 ms, total: 1min 6s
Wall time: 35.1 s


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
#Creating Train and Test Paragraph Vectors for Distributed Bag of Words Model
Y_train_agr, X_train_agr = vec_for_learning(model_dbow, train_tagged_agr)
Y_test_agr, X_test_agr = vec_for_learning(model_dbow, test_tagged_agr)

In [ ]:
# Results for Logistic Regression 

logreg.fit(X_train_agr, Y_train_agr)
Y_lg_pred_agr = logreg.predict(X_test_agr)
print('Testing accuracy while using Logistic Regression %s' % accuracy_score(Y_test_agr, Y_lg_pred_agr))
print('Testing F1 score while using Logistic Regression: {}'.format(f1_score(Y_test_agr, Y_lg_pred_agr, average='weighted')))

Testing accuracy while using Logistic Regression 0.5506072874493927
Testing F1 score while using Logistic Regression: 0.5501740874539145


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for SGD Classifier

sgd.fit(X_train_agr, Y_train_agr)
Y_sgd_pred_agr = sgd.predict(X_test_agr)
print('Testing accuracy while using SGD Classifier %s' % accuracy_score(Y_test_agr, Y_sgd_pred_agr))
print('Testing F1 score while using SGD Classifier: {}'.format(f1_score(Y_test_agr, Y_sgd_pred_agr, average='weighted')))

Testing accuracy while using SGD Classifier 0.520242914979757
Testing F1 score while using SGD Classifier: 0.4631701179264753


In [ ]:
# Results for Passive Aggressive Classifier

pac.fit(X_train_agr, Y_train_agr)
Y_pac_pred_agr = pac.predict(X_test_agr)
print('Testing accuracy while using Passive Aggressive Classifier %s' % accuracy_score(Y_test_agr, Y_pac_pred_agr))
print('Testing F1 score while using Passive Aggressive Classifier: {}'.format(f1_score(Y_test_agr, Y_pac_pred_agr, average='weighted')))

Testing accuracy while using Passive Aggressive Classifier 0.4979757085020243
Testing F1 score while using Passive Aggressive Classifier: 0.45504085279753953


In [ ]:
# Results for Ridge Classifier

rc.fit(X_train_agr, Y_train_agr)
Y_rc_pred_agr = rc.predict(X_test_agr)
print('Testing accuracy while using Ridge Classifier %s' % accuracy_score(Y_test_agr, Y_rc_pred_agr))
print('Testing F1 score while using Ridge Classifier: {}'.format(f1_score(Y_test_agr, Y_rc_pred_agr, average='weighted')))

Testing accuracy while using Ridge Classifier 0.5303643724696356
Testing F1 score while using Ridge Classifier: 0.5278161510886995


In [ ]:
# Results for Support Vector Classifier

svc.fit(X_train_agr, Y_train_agr)
Y_svc_pred_agr = svc.predict(X_test_agr)
print('Testing accuracy while using Support Vector Classifier %s' % accuracy_score(Y_test_agr, Y_svc_pred_agr))
print('Testing F1 score while using Support Vector Classifier: {}'.format(f1_score(Y_test_agr, Y_svc_pred_agr, average='weighted')))

Testing accuracy while using Support Vector Classifier 0.5323886639676113
Testing F1 score while using Support Vector Classifier: 0.4567815204934318


In [ ]:
# Results for XGBoost Algorithm
xgbc.fit(np.array(X_train_agr), Y_train_agr)
xgb_predict_agr = xgbc.predict(np.array(X_test_agr))

print('Testing accuracy while using XGBoost Algorithm %s' % accuracy_score(Y_test_agr,xgb_predict_agr))
print('Testing F1 score while using XGBoost Algorithm: {}'.format(f1_score(Y_test_agr, xgb_predict_agr, average='weighted')))

Testing accuracy while using XGBoost Algorithm 0.5344129554655871
Testing F1 score while using XGBoost Algorithm: 0.5339641446594611


In [ ]:
hard_voting_agr = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='hard')
soft_voting_agr = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='soft')

In [ ]:
hard_voting_agr.fit(np.array(X_train_agr), Y_train_agr)
soft_voting_agr.fit(np.array(X_train_agr), Y_train_agr)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=100000.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=750,
                                                 multi_class='warn', n_jobs=1,
                                                 penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('SGD',
                              SGDClassifier(alpha=0.0001, average=False,
                                            class_weight=None,
                                            early_stop...
                                  random_state=None, shri

In [ ]:
Y_hard_voting_agr = hard_voting_agr.predict(np.array(X_test_agr))
Y_soft_voting_agr = soft_voting_agr.predict(np.array(X_test_agr))

In [ ]:
print('Testing accuracy while using Ensemble Model with Hard Voting %s' % accuracy_score(Y_test_agr,Y_hard_voting_agr))
print('Testing F1 score while using Ensemble Model with Hard Voting: {}'.format(f1_score(Y_test_agr, Y_hard_voting_agr, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting 0.5364372469635628
Testing F1 score while using Ensemble Model with Hard Voting: 0.5333748407153931


In [ ]:
print('Testing accuracy while using Ensemble Model with Soft Voting %s' % accuracy_score(Y_test_agr,Y_soft_voting_agr))
print('Testing F1 score while using Ensemble Model with S Voting: {}'.format(f1_score(Y_test_agr, Y_soft_voting_agr, average='weighted')))

Testing accuracy while using Ensemble Model with Soft Voting 0.5344129554655871
Testing F1 score while using Ensemble Model with S Voting: 0.53001838582484


**Out of all the models shown above, Logistic Regression has the best Accuracy and F1-Score, 55.06% and 55.01% respectively. However, for cAGR, the State-of-the-Art model has an accuracy of 56.71%. Thus, our model fails to match the SOTA Accuracy, by 1.65%.**





**We will try and predict the personality trait of Conscientiouosness(cCON) for the given dataset**



In [ ]:
# Training and Test Documents for Conscientiouosness
train_tagged_con = train.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cCON]), axis=1)
test_tagged_con = test.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cCON]), axis=1)

In [ ]:
train_tagged_con.values[30]

TaggedDocument(words=['feel', 'really', 'crappy', 'today', 'get', 'enough', 'sleep', 'last', 'night', 'think', 'part', 'texas', 'loss', 'saturday', 'especially', 'way', 'loss', 'least', 'cowboys', 'last', 'night', 'far', 'home', 'sick', 'might', 'roommate', 'go', 'high', 'school', 'also', 'able', 'talk', 'parent', 'lot', 'instant', 'chat', 'mom', 'wnts', 'get', 'web', 'cam', 'actually', 'see', 'think', 'hard', 'time', 'leave', 'first', 'child', 'leave', 'home', 'the', 'network', 'really', 'cool', 'the', 'connect', 'rat', 'really', 'fast', 'crummy', 'modem', 'home', 'dad', 'go', 'get', 'dsl', 'live', 'far', 'central', 'office', 'suck', 'really', 'hungry', 'hungry', 'morning', 'halfway', 'class', 'stomach', 'start', 'hurt', 'probably', 'use', 'sleep', 'late', 'eat', 'breakfast', 'min', 'take', 'english', 'year', 'good', 'like', 'write', 'paper', 'much', 'you', 'might', 'write', 'paper', 'feel', 'perfectly', 'valid', 'turn', 'wrong', 'yet', 'put', 'much', 'work', 'min', 'milk', 'taste', '

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=230, negative=5, hs=1, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged_con.values)])

100%|██████████| 1973/1973 [00:00<00:00, 1009313.55it/s]


In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged_con.values)]), total_examples=len(train_tagged_con.values), epochs=1)
    model_dbow.alpha -= 0.02
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1973/1973 [00:00<00:00, 1494017.29it/s]


CPU times: user 1min 11s, sys: 468 ms, total: 1min 12s
Wall time: 38 s


In [ ]:
#Creating Train and Test Paragraph Vectors for Distributed Bag of Words Model
Y_train_con, X_train_con = vec_for_learning(model_dbow, train_tagged_con)
Y_test_con, X_test_con = vec_for_learning(model_dbow, test_tagged_con)

In [ ]:
# Results for Logistic Regression 

logreg.fit(X_train_con, Y_train_con)
Y_lg_pred_con = logreg.predict(X_test_con)
print('Testing accuracy while using Logistic Regression %s' % accuracy_score(Y_test_con, Y_lg_pred_con))
print('Testing F1 score while using Logistic Regression: {}'.format(f1_score(Y_test_con, Y_lg_pred_con, average='weighted')))

Testing accuracy while using Logistic Regression 0.5222672064777328
Testing F1 score while using Logistic Regression: 0.5229253593430317


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for SGD Classifier

sgd.fit(X_train_con, Y_train_con)
Y_sgd_pred_con = sgd.predict(X_test_con)
print('Testing accuracy while using SGD Classifier %s' % accuracy_score(Y_test_con, Y_sgd_pred_con))
print('Testing F1 score while using SGD Classifier: {}'.format(f1_score(Y_test_con, Y_sgd_pred_con, average='weighted')))

Testing accuracy while using SGD Classifier 0.5526315789473685
Testing F1 score while using SGD Classifier: 0.5450671238878249


In [ ]:
# Results for Passive Aggressive Classifier

pac.fit(X_train_con, Y_train_con)
Y_pac_pred_con = pac.predict(X_test_con)
print('Testing accuracy while using Passive Aggressive Classifier %s' % accuracy_score(Y_test_con, Y_pac_pred_con))
print('Testing F1 score while using Passive Aggressive Classifier: {}'.format(f1_score(Y_test_con, Y_pac_pred_con, average='weighted')))

Testing accuracy while using Passive Aggressive Classifier 0.48785425101214575
Testing F1 score while using Passive Aggressive Classifier: 0.4587667874662192


In [ ]:
# Results for Ridge Classifier

rc.fit(X_train_con, Y_train_con)
Y_rc_pred_con = rc.predict(X_test_con)
print('Testing accuracy while using Ridge Classifier %s' % accuracy_score(Y_test_con, Y_rc_pred_con))
print('Testing F1 score while using Ridge Classifier: {}'.format(f1_score(Y_test_con, Y_rc_pred_con, average='weighted')))

Testing accuracy while using Ridge Classifier 0.5546558704453441
Testing F1 score while using Ridge Classifier: 0.5552710169910268


In [ ]:
# Results for Support Vector Classifier

svc.fit(X_train_con, Y_train_con)
Y_svc_pred_con = svc.predict(X_test_con)
print('Testing accuracy while using Support Vector Classifier %s' % accuracy_score(Y_test_con, Y_svc_pred_con))
print('Testing F1 score while using Support Vector Classifier: {}'.format(f1_score(Y_test_con, Y_svc_pred_con, average='weighted')))

Testing accuracy while using Support Vector Classifier 0.548582995951417
Testing F1 score while using Support Vector Classifier: 0.5417892887257325


In [ ]:
# Results for XGBoost Algorithm
xgbc.fit(np.array(X_train_con), Y_train_con)
xgb_predict_con = xgbc.predict(np.array(X_test_con))

print('Testing accuracy while using XGBoost Algorithm %s' % accuracy_score(Y_test_con,xgb_predict_con))
print('Testing F1 score while using XGBoost Algorithm: {}'.format(f1_score(Y_test_con, xgb_predict_con, average='weighted')))

Testing accuracy while using XGBoost Algorithm 0.5182186234817814
Testing F1 score while using XGBoost Algorithm: 0.5174532963898655


In [ ]:
hard_voting_con = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='hard')
soft_voting_con = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='soft')

In [ ]:
hard_voting_con.fit(np.array(X_train_con), Y_train_con)
soft_voting_con.fit(np.array(X_train_con), Y_train_con)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=100000.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=750,
                                                 multi_class='warn', n_jobs=1,
                                                 penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('SGD',
                              SGDClassifier(alpha=0.0001, average=False,
                                            class_weight=None,
                                            early_stop...
                                  random_state=None, shri

In [ ]:
Y_hard_voting_con = hard_voting_con.predict(np.array(X_test_con))
Y_soft_voting_con = soft_voting_con.predict(np.array(X_test_con))

In [ ]:
print('Testing accuracy while using Ensemble Model with Hard Voting %s' % accuracy_score(Y_test_con,Y_hard_voting_con))
print('Testing F1 score while using Ensemble Model with Hard Voting: {}'.format(f1_score(Y_test_con, Y_hard_voting_con, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting 0.5364372469635628
Testing F1 score while using Ensemble Model with Hard Voting: 0.533296067144104


In [ ]:
print('Testing accuracy while using Ensemble Model with Soft Voting %s' % accuracy_score(Y_test_con,Y_soft_voting_con))
print('Testing F1 score while using Ensemble Model with S Voting: {}'.format(f1_score(Y_test_con, Y_soft_voting_con, average='weighted')))

Testing accuracy while using Ensemble Model with Soft Voting 0.5222672064777328
Testing F1 score while using Ensemble Model with S Voting: 0.5086329149346224


**Out of all the models shown above, Ridge Classifier has the best Accuracy and F1-Score, 55.46% and 55.52% respectively. However, for cCon, the State-of-the-Art model has an accuracy of 57.30%. Thus, our model fails to match the SOTA Accuracy, by 1.84%.**

**We will try and predict the personality trait of Openness(cOPN) for the given dataset**

In [ ]:
# Training and Test Documents for Openness
train_tagged_opn = train.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cOPN]), axis=1)
test_tagged_opn = test.apply(lambda r: TaggedDocument(words=normalize_text(r['TEXT']), tags=[r.cOPN]), axis=1)

In [ ]:
train_tagged_opn.values[30]

TaggedDocument(words=['feel', 'really', 'crappy', 'today', 'get', 'enough', 'sleep', 'last', 'night', 'think', 'part', 'texas', 'loss', 'saturday', 'especially', 'way', 'loss', 'least', 'cowboys', 'last', 'night', 'far', 'home', 'sick', 'might', 'roommate', 'go', 'high', 'school', 'also', 'able', 'talk', 'parent', 'lot', 'instant', 'chat', 'mom', 'wnts', 'get', 'web', 'cam', 'actually', 'see', 'think', 'hard', 'time', 'leave', 'first', 'child', 'leave', 'home', 'the', 'network', 'really', 'cool', 'the', 'connect', 'rat', 'really', 'fast', 'crummy', 'modem', 'home', 'dad', 'go', 'get', 'dsl', 'live', 'far', 'central', 'office', 'suck', 'really', 'hungry', 'hungry', 'morning', 'halfway', 'class', 'stomach', 'start', 'hurt', 'probably', 'use', 'sleep', 'late', 'eat', 'breakfast', 'min', 'take', 'english', 'year', 'good', 'like', 'write', 'paper', 'much', 'you', 'might', 'write', 'paper', 'feel', 'perfectly', 'valid', 'turn', 'wrong', 'yet', 'put', 'much', 'work', 'min', 'milk', 'taste', '

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=230, negative=5, hs=1, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged_opn.values)])

100%|██████████| 1973/1973 [00:00<00:00, 1358621.21it/s]


In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged_opn.values)]), total_examples=len(train_tagged_opn.values), epochs=1)
    model_dbow.alpha -= 0.02
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1973/1973 [00:00<00:00, 510793.27it/s]


CPU times: user 1min 7s, sys: 443 ms, total: 1min 8s
Wall time: 36 s


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
#Creating Train and Test Paragraph Vectors for Distributed Bag of Words Model
Y_train_opn, X_train_opn = vec_for_learning(model_dbow, train_tagged_opn)
Y_test_opn, X_test_opn = vec_for_learning(model_dbow, test_tagged_opn)

In [ ]:
# Results for Logistic Regression 

logreg.fit(X_train_opn, Y_train_opn)
Y_lg_pred_opn = logreg.predict(X_test_opn)
print('Testing accuracy while using Logistic Regression %s' % accuracy_score(Y_test_opn, Y_lg_pred_opn))
print('Testing F1 score while using Logistic Regression: {}'.format(f1_score(Y_test_opn, Y_lg_pred_opn, average='weighted')))

Testing accuracy while using Logistic Regression 0.5526315789473685
Testing F1 score while using Logistic Regression: 0.5520039840421789


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
# Results for SGD Classifier

sgd.fit(X_train_opn, Y_train_opn)
Y_sgd_pred_opn = sgd.predict(X_test_opn)
print('Testing accuracy while using SGD Classifier %s' % accuracy_score(Y_test_opn, Y_sgd_pred_opn))
print('Testing F1 score while using SGD Classifier: {}'.format(f1_score(Y_test_opn, Y_sgd_pred_opn, average='weighted')))

Testing accuracy while using SGD Classifier 0.548582995951417
Testing F1 score while using SGD Classifier: 0.537455397950008


In [ ]:
# Results for Passive Aggressive Classifier

pac.fit(X_train_opn, Y_train_opn)
Y_pac_pred_opn = pac.predict(X_test_opn)
print('Testing accuracy while using Passive Aggressive Classifier %s' % accuracy_score(Y_test_opn, Y_pac_pred_opn))
print('Testing F1 score while using Passive Aggressive Classifier: {}'.format(f1_score(Y_test_opn, Y_pac_pred_opn, average='weighted')))

Testing accuracy while using Passive Aggressive Classifier 0.5060728744939271
Testing F1 score while using Passive Aggressive Classifier: 0.47152381742208266


In [ ]:
# Results for Ridge Classifier

rc.fit(X_train_opn, Y_train_opn)
Y_rc_pred_opn = rc.predict(X_test_opn)
print('Testing accuracy while using Ridge Classifier %s' % accuracy_score(Y_test_opn, Y_rc_pred_opn))
print('Testing F1 score while using Ridge Classifier: {}'.format(f1_score(Y_test_opn, Y_rc_pred_opn, average='weighted')))

Testing accuracy while using Ridge Classifier 0.5769230769230769
Testing F1 score while using Ridge Classifier: 0.5768031940955102


In [ ]:
# Results for Support Vector Classifier

svc.fit(X_train_opn, Y_train_opn)
Y_svc_pred_opn = svc.predict(X_test_opn)
print('Testing accuracy while using Support Vector Classifier %s' % accuracy_score(Y_test_opn, Y_svc_pred_opn))
print('Testing F1 score while using Support Vector Classifier: {}'.format(f1_score(Y_test_opn, Y_svc_pred_opn, average='weighted')))

Testing accuracy while using Support Vector Classifier 0.5688259109311741
Testing F1 score while using Support Vector Classifier: 0.567708295012964


In [ ]:
# Results for XGBoost Algorithm
xgbc.fit(np.array(X_train_opn), Y_train_opn)
xgb_predict_opn = xgbc.predict(np.array(X_test_opn))

print('Testing accuracy while using XGBoost Algorithm %s' % accuracy_score(Y_test_opn,xgb_predict_opn))
print('Testing F1 score while using XGBoost Algorithm: {}'.format(f1_score(Y_test_opn, xgb_predict_opn, average='weighted')))

Testing accuracy while using XGBoost Algorithm 0.5647773279352226
Testing F1 score while using XGBoost Algorithm: 0.564868391030462


In [ ]:
hard_voting_opn = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='hard')
soft_voting_opn = VotingClassifier(estimators=[('LR', logreg), ('SGD', sgd), ('XGBC', xgbc), ('SVC', svc), ('LGCV', lgcv)], voting='soft')

In [ ]:
hard_voting_opn.fit(np.array(X_train_opn), Y_train_opn)
soft_voting_opn.fit(np.array(X_train_opn), Y_train_opn)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=100000.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=750,
                                                 multi_class='warn', n_jobs=1,
                                                 penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('SGD',
                              SGDClassifier(alpha=0.0001, average=False,
                                            class_weight=None,
                                            early_stop...
                                  random_state=None, shri

In [ ]:
Y_hard_voting_opn = hard_voting_opn.predict(np.array(X_test_opn))
Y_soft_voting_opn = soft_voting_opn.predict(np.array(X_test_opn))

In [ ]:
print('Testing accuracy while using Ensemble Model with Hard Voting %s' % accuracy_score(Y_test_opn,Y_hard_voting_opn))
print('Testing F1 score while using Ensemble Model with Hard Voting: {}'.format(f1_score(Y_test_opn, Y_hard_voting_opn, average='weighted')))

Testing accuracy while using Ensemble Model with Hard Voting 0.5688259109311741
Testing F1 score while using Ensemble Model with Hard Voting: 0.5647015668978038


In [ ]:
print('Testing accuracy while using Ensemble Model with Soft Voting %s' % accuracy_score(Y_test_opn,Y_soft_voting_opn))
print('Testing F1 score while using Ensemble Model with S Voting: {}'.format(f1_score(Y_test_opn, Y_soft_voting_opn, average='weighted')))

Testing accuracy while using Ensemble Model with Soft Voting 0.5445344129554656
Testing F1 score while using Ensemble Model with S Voting: 0.5285993990883741


**Out of all the models shown above, Ridge Classifier has the best Accuracy and F1-Score, 57.69% and 57.68% respectively. However, for cCon, the State-of-the-Art model has an accuracy of 57.30%. Thus, our model fails to match the SOTA Accuracy, by 4.99%.**